# IG 爬蟲:
## 鏈接： [IG登入教學](https://medium.com/marketingdatascience/%E8%B7%9F%E8%91%97ig%E6%BD%AE%E6%B5%81%E4%BE%86%E7%88%AC%E8%9F%B2-%E7%94%A8selenium%E5%B8%B6%E6%82%A8%E8%87%AA%E5%8B%95%E7%99%BB%E5%85%A5-ig-%E7%B3%BB%E5%88%971-%E9%99%84python%E7%A8%8B%E5%BC%8F%E7%A2%BC-846d57f73cac)

## 鏈接：[IG爬蟲教學](https://medium.com/marketingdatascience/%E8%B7%9F%E8%91%97ig%E6%BD%AE%E6%B5%81%E4%BE%86%E7%88%AC%E8%9F%B2-%E5%A6%82%E4%BD%95%E7%88%AC%E5%8F%96ig%E8%B2%BC%E6%96%87%E7%9F%AD%E9%80%A3%E7%B5%90-%E7%B3%BB%E5%88%972-%E9%99%84python%E7%A8%8B%E5%BC%8F%E7%A2%BC-465b7f00eeee)


### 載入套件

In [ ]:
#! pip install selenium
#! pip install beautifulsoup4
#! pip install chromedriver-autoinstaller

In [1]:
#from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as Soup
from selenium.webdriver.support.wait import WebDriverWait #顯示等待
from selenium.webdriver.support import expected_conditions as EC #主要用於對頁面元素的載入進行判斷————這裡和 WebDriverWait結合運用
from selenium.common.exceptions import NoSuchElementException #找原素找不到就會拋出 NoSuchEkementException
from selenium.webdriver.common.keys import Keys #模擬鍵盤的按鍵
from selenium.webdriver.common.action_chains import ActionChains #模擬鼠標拖拽、點擊等動作
from selenium.webdriver.common.by import By #類似 EC
from selenium.webdriver import DesiredCapabilities #負責啟動服務端時的參數設置
from pandas.core.frame import DataFrame
import pandas as pd
import time

## 下載最新的chromedriver.exe

In [3]:
#使用SSL module把證書驗證改成不需要驗證
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


# 下載最新的chromedriver.exe
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'C:\\Anaconda3\\envs\\NTPU_project\\lib\\site-packages\\chromedriver_autoinstaller\\105\\chromedriver.exe'

## 自動填入賬號密碼，且點擊登入

In [4]:
browser = webdriver.Chrome()
url = 'https://www.instagram.com/accounts/login/'  

# ------ 到要去的網站 ------
browser.get(url) 

# ------ 輸入帳號與密碼 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.NAME, 'username')))

# ------ 網頁元素定位 ------
username_input = browser.find_elements_by_name('username')[0]
password_input = browser.find_elements_by_name('password')[0]
print("inputing username and password...")

# ------ 輸入帳號密碼 ------！！！賬號和密碼要換成自己的哦！！！
username_input.send_keys("tiffany.hhh")
password_input.send_keys("aniki13liang17")

# ------ 登入 ------
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH,
'//*[@id="loginForm"]/div/div[3]')))
# ------ 網頁元素定位 ------
login_click = browser.find_elements_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div')[0]
# ------ 點擊登入鍵 ------
login_click.click()

inputing username and password...


## 到要去的頁面

In [5]:
search = '2022健康臺北尋寶趣'

In [7]:
# 切換頁面
url = f'https://www.instagram.com/explore/tags/{search}/'
browser.get(url)

## 爬網頁名

In [19]:
from bs4 import BeautifulSoup

htmltext = browser.page_source
soup = BeautifulSoup(htmltext,"lxml")

# 可能需要改網頁碼
# 爬ig粉專
siteName = soup.find_all('h2', '_7UhW9 fKFbl yUEEX KV-D4 fDxYl')[0].text
print(siteName)

niusnews


# 收集IG貼文+留言


## 按暫停就等於結束哦！！！

In [20]:
import time


postL = [] #把每篇內容存入空間

def click_expand_posts():
    button = True
    while button: # 如果更多留言的按鈕存在
        try:
            # 等待直到Button出現 
            # (browser,timeout,poll_frequency)裡面的數字可以換哦——原本是等待時間15秒，默認0.5秒搜一次
            more_ele =  WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.XPATH,'/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/li/div/button')))
            more_ele.click()
            print("更多")
        
        except:
            button = False
            print("這篇已經沒有更多留言嘍")
            
            htmltext = browser.page_source
            soup = BeautifulSoup(htmltext,"lxml")
            postL.extend(soup.find_all('div','C4VMK'))
            # 總共讀到幾篇文(貼文+留言)
            postList= postL[:len(postL)]
            print( "******已讀取" ,len(postList),"篇貼文+留言******")
            
            next_click = browser.find_element(By.XPATH,'//button[@class="wpO6b  "]//*[@aria-label="下一步"]')
            next_click.click()
            print("執行中")
            time.sleep(0.2) #裡面數字可以換哦————原本是暫停 2秒
            
timespend = 1    
time_start = time.time() #開始計時
    
#while timespend< 150:
while True:
    try:
        click_expand_posts()
        time_end = time.time()
        timespend = (time_end- time_start)/60
        print('spend'+str(round(timespend, 2))+ 'min')       

    #當按暫停鍵時---要連續按哦！！一次是下一頁
    except KeyboardInterrupt:
        print("force to stop by user")
        break

這篇已經沒有更多留言嘍
******已讀取 1 篇貼文+留言******
執行中
spend0.26min
這篇已經沒有更多留言嘍
******已讀取 2 篇貼文+留言******
執行中
spend0.52min
這篇已經沒有更多留言嘍
******已讀取 4 篇貼文+留言******
執行中
spend0.78min
這篇已經沒有更多留言嘍
******已讀取 5 篇貼文+留言******
執行中
spend1.04min
這篇已經沒有更多留言嘍
******已讀取 12 篇貼文+留言******
執行中
spend1.31min
這篇已經沒有更多留言嘍
******已讀取 15 篇貼文+留言******
執行中
spend1.57min
這篇已經沒有更多留言嘍
******已讀取 17 篇貼文+留言******
執行中
spend1.82min
這篇已經沒有更多留言嘍
******已讀取 19 篇貼文+留言******
執行中
spend2.08min
這篇已經沒有更多留言嘍
******已讀取 30 篇貼文+留言******
執行中
spend2.35min
這篇已經沒有更多留言嘍
******已讀取 32 篇貼文+留言******
執行中
spend2.61min
更多
更多
更多
更多
這篇已經沒有更多留言嘍
******已讀取 75 篇貼文+留言******
執行中
spend2.95min
這篇已經沒有更多留言嘍
******已讀取 77 篇貼文+留言******
執行中
spend3.21min
這篇已經沒有更多留言嘍
******已讀取 78 篇貼文+留言******
執行中
spend3.47min
這篇已經沒有更多留言嘍
******已讀取 79 篇貼文+留言******
執行中
spend3.73min
這篇已經沒有更多留言嘍
******已讀取 81 篇貼文+留言******
執行中
spend3.99min
這篇已經沒有更多留言嘍
******已讀取 82 篇貼文+留言******
執行中
spend4.25min
這篇已經沒有更多留言嘍
******已讀取 83 篇貼文+留言******
執行中
spend4.52min
更多
這篇已經沒有更多留言嘍
******已讀取 103 篇貼文+留言******
執行中
spend4.8min

執行中
spend38.53min
這篇已經沒有更多留言嘍
******已讀取 1783 篇貼文+留言******
執行中
spend38.79min
這篇已經沒有更多留言嘍
******已讀取 1785 篇貼文+留言******
執行中
spend39.05min
這篇已經沒有更多留言嘍
******已讀取 1787 篇貼文+留言******
執行中
spend39.31min
這篇已經沒有更多留言嘍
******已讀取 1789 篇貼文+留言******
執行中
spend39.57min
這篇已經沒有更多留言嘍
******已讀取 1793 篇貼文+留言******
執行中
spend39.83min
這篇已經沒有更多留言嘍
******已讀取 1799 篇貼文+留言******
執行中
spend40.1min
這篇已經沒有更多留言嘍
******已讀取 1800 篇貼文+留言******
執行中
spend40.36min
這篇已經沒有更多留言嘍
******已讀取 1803 篇貼文+留言******
執行中
spend40.62min
這篇已經沒有更多留言嘍
******已讀取 1807 篇貼文+留言******
執行中
spend40.88min
這篇已經沒有更多留言嘍
******已讀取 1812 篇貼文+留言******
執行中
spend41.14min
這篇已經沒有更多留言嘍
******已讀取 1824 篇貼文+留言******
執行中
spend41.4min
這篇已經沒有更多留言嘍
******已讀取 1825 篇貼文+留言******
執行中
spend41.66min
這篇已經沒有更多留言嘍
******已讀取 1826 篇貼文+留言******
執行中
spend41.93min
這篇已經沒有更多留言嘍
******已讀取 1827 篇貼文+留言******
執行中
spend42.19min
這篇已經沒有更多留言嘍
******已讀取 1837 篇貼文+留言******
執行中
spend42.45min
這篇已經沒有更多留言嘍
******已讀取 1838 篇貼文+留言******
執行中
spend42.71min
這篇已經沒有更多留言嘍
******已讀取 1839 篇貼文+留言******
執行中
spend42.97mi

spend75.71min
這篇已經沒有更多留言嘍
******已讀取 1965 篇貼文+留言******
執行中
spend75.98min
這篇已經沒有更多留言嘍
******已讀取 1966 篇貼文+留言******
執行中
spend76.24min
這篇已經沒有更多留言嘍
******已讀取 1967 篇貼文+留言******
執行中
spend76.5min
這篇已經沒有更多留言嘍
******已讀取 1968 篇貼文+留言******
執行中
spend76.76min
這篇已經沒有更多留言嘍
******已讀取 1969 篇貼文+留言******
執行中
spend77.02min
這篇已經沒有更多留言嘍
******已讀取 1970 篇貼文+留言******
執行中
spend77.29min
這篇已經沒有更多留言嘍
******已讀取 1971 篇貼文+留言******
執行中
spend77.55min
這篇已經沒有更多留言嘍
******已讀取 1972 篇貼文+留言******
執行中
spend77.81min
這篇已經沒有更多留言嘍
******已讀取 1973 篇貼文+留言******
執行中
spend78.07min
這篇已經沒有更多留言嘍
******已讀取 1974 篇貼文+留言******
執行中
spend78.33min
這篇已經沒有更多留言嘍
******已讀取 1975 篇貼文+留言******
執行中
spend78.59min
這篇已經沒有更多留言嘍
******已讀取 1976 篇貼文+留言******
執行中
spend78.85min
這篇已經沒有更多留言嘍
******已讀取 1977 篇貼文+留言******
執行中
spend79.11min
這篇已經沒有更多留言嘍
******已讀取 1978 篇貼文+留言******
執行中
spend79.37min
這篇已經沒有更多留言嘍
******已讀取 1979 篇貼文+留言******
執行中
spend79.63min
這篇已經沒有更多留言嘍
******已讀取 1980 篇貼文+留言******
執行中
spend79.89min
這篇已經沒有更多留言嘍
******已讀取 1981 篇貼文+留言******
執行中
spend80.15min
這

spend112.68min
這篇已經沒有更多留言嘍
******已讀取 2106 篇貼文+留言******
執行中
spend112.94min
這篇已經沒有更多留言嘍
******已讀取 2107 篇貼文+留言******
執行中
spend113.2min
這篇已經沒有更多留言嘍
******已讀取 2108 篇貼文+留言******
執行中
spend113.47min
這篇已經沒有更多留言嘍
******已讀取 2109 篇貼文+留言******
執行中
spend113.73min
這篇已經沒有更多留言嘍
******已讀取 2110 篇貼文+留言******
執行中
spend113.99min
這篇已經沒有更多留言嘍
******已讀取 2111 篇貼文+留言******
執行中
spend114.25min
這篇已經沒有更多留言嘍
******已讀取 2112 篇貼文+留言******
執行中
spend114.52min
這篇已經沒有更多留言嘍
******已讀取 2113 篇貼文+留言******
執行中
spend114.78min
這篇已經沒有更多留言嘍
******已讀取 2114 篇貼文+留言******
執行中
spend115.04min
這篇已經沒有更多留言嘍
******已讀取 2115 篇貼文+留言******
執行中
spend115.3min
這篇已經沒有更多留言嘍
******已讀取 2116 篇貼文+留言******
執行中
spend115.57min
這篇已經沒有更多留言嘍
******已讀取 2117 篇貼文+留言******
執行中
spend115.83min
這篇已經沒有更多留言嘍
******已讀取 2118 篇貼文+留言******
執行中
spend116.09min
這篇已經沒有更多留言嘍
******已讀取 2119 篇貼文+留言******
執行中
spend116.35min
這篇已經沒有更多留言嘍
******已讀取 2120 篇貼文+留言******
執行中
spend116.61min
這篇已經沒有更多留言嘍
******已讀取 2121 篇貼文+留言******
執行中
spend116.88min
這篇已經沒有更多留言嘍
******已讀取 2122 篇貼文+留言******
執行中

這篇已經沒有更多留言嘍
******已讀取 2245 篇貼文+留言******
執行中
spend149.46min
這篇已經沒有更多留言嘍
******已讀取 2246 篇貼文+留言******
執行中
spend149.73min
這篇已經沒有更多留言嘍
******已讀取 2247 篇貼文+留言******
執行中
spend149.99min
這篇已經沒有更多留言嘍
******已讀取 2248 篇貼文+留言******
執行中
spend150.25min
這篇已經沒有更多留言嘍
******已讀取 2249 篇貼文+留言******
執行中
spend150.51min
這篇已經沒有更多留言嘍
******已讀取 2250 篇貼文+留言******
執行中
spend150.77min
這篇已經沒有更多留言嘍
******已讀取 2251 篇貼文+留言******
執行中
spend151.03min
這篇已經沒有更多留言嘍
******已讀取 2252 篇貼文+留言******
執行中
spend151.3min
這篇已經沒有更多留言嘍
******已讀取 2253 篇貼文+留言******
執行中
spend151.56min
這篇已經沒有更多留言嘍
******已讀取 2254 篇貼文+留言******
執行中
spend151.82min
這篇已經沒有更多留言嘍
******已讀取 2255 篇貼文+留言******
執行中
spend152.08min
這篇已經沒有更多留言嘍
******已讀取 2256 篇貼文+留言******
執行中
spend152.35min
這篇已經沒有更多留言嘍
******已讀取 2257 篇貼文+留言******
執行中
spend152.61min
這篇已經沒有更多留言嘍
******已讀取 2258 篇貼文+留言******
執行中
spend152.87min
這篇已經沒有更多留言嘍
******已讀取 2259 篇貼文+留言******
執行中
spend153.13min
這篇已經沒有更多留言嘍
******已讀取 2260 篇貼文+留言******
執行中
spend153.4min
這篇已經沒有更多留言嘍
******已讀取 2261 篇貼文+留言******
執行中
spend153.66min

執行中
spend186.08min
這篇已經沒有更多留言嘍
******已讀取 2385 篇貼文+留言******
執行中
spend186.34min
這篇已經沒有更多留言嘍
******已讀取 2386 篇貼文+留言******
執行中
spend186.6min
這篇已經沒有更多留言嘍
******已讀取 2387 篇貼文+留言******
執行中
spend186.86min
這篇已經沒有更多留言嘍
******已讀取 2388 篇貼文+留言******
執行中
spend187.12min
這篇已經沒有更多留言嘍
******已讀取 2389 篇貼文+留言******
執行中
spend187.39min
這篇已經沒有更多留言嘍
******已讀取 2390 篇貼文+留言******
執行中
spend187.65min
這篇已經沒有更多留言嘍
******已讀取 2391 篇貼文+留言******
執行中
spend187.91min
這篇已經沒有更多留言嘍
******已讀取 2392 篇貼文+留言******
執行中
spend188.17min
這篇已經沒有更多留言嘍
******已讀取 2393 篇貼文+留言******
執行中
spend188.44min
這篇已經沒有更多留言嘍
******已讀取 2394 篇貼文+留言******
執行中
spend188.7min
這篇已經沒有更多留言嘍
******已讀取 2395 篇貼文+留言******
執行中
spend188.96min
這篇已經沒有更多留言嘍
******已讀取 2396 篇貼文+留言******
執行中
spend189.23min
這篇已經沒有更多留言嘍
******已讀取 2397 篇貼文+留言******
執行中
spend189.49min
這篇已經沒有更多留言嘍
******已讀取 2398 篇貼文+留言******
執行中
spend189.75min
這篇已經沒有更多留言嘍
******已讀取 2399 篇貼文+留言******
執行中
spend190.02min
這篇已經沒有更多留言嘍
******已讀取 2400 篇貼文+留言******
執行中
spend190.28min
這篇已經沒有更多留言嘍
******已讀取 2401 篇貼文+留言******

spend223.16min
這篇已經沒有更多留言嘍
******已讀取 2525 篇貼文+留言******
執行中
spend223.42min
這篇已經沒有更多留言嘍
******已讀取 2526 篇貼文+留言******
執行中
spend223.69min
這篇已經沒有更多留言嘍
******已讀取 2527 篇貼文+留言******
執行中
spend223.95min
這篇已經沒有更多留言嘍
******已讀取 2528 篇貼文+留言******
執行中
spend224.21min
這篇已經沒有更多留言嘍
******已讀取 2529 篇貼文+留言******
執行中
spend224.47min
這篇已經沒有更多留言嘍
******已讀取 2530 篇貼文+留言******
執行中
spend224.74min
這篇已經沒有更多留言嘍
******已讀取 2531 篇貼文+留言******
執行中
spend225.0min
這篇已經沒有更多留言嘍
******已讀取 2532 篇貼文+留言******
執行中
spend225.26min
這篇已經沒有更多留言嘍
******已讀取 2533 篇貼文+留言******
執行中
spend225.52min
這篇已經沒有更多留言嘍
******已讀取 2534 篇貼文+留言******
執行中
spend225.79min
這篇已經沒有更多留言嘍
******已讀取 2535 篇貼文+留言******
執行中
spend226.05min
這篇已經沒有更多留言嘍
******已讀取 2536 篇貼文+留言******
執行中
spend226.31min
這篇已經沒有更多留言嘍
******已讀取 2537 篇貼文+留言******
執行中
spend226.57min
這篇已經沒有更多留言嘍
******已讀取 2538 篇貼文+留言******
執行中
spend226.84min
這篇已經沒有更多留言嘍
******已讀取 2539 篇貼文+留言******
執行中
spend227.1min
這篇已經沒有更多留言嘍
******已讀取 2540 篇貼文+留言******
執行中
spend227.36min
這篇已經沒有更多留言嘍
******已讀取 2541 篇貼文+留言******
執行中

這篇已經沒有更多留言嘍
******已讀取 2664 篇貼文+留言******
執行中
spend259.91min
這篇已經沒有更多留言嘍
******已讀取 2665 篇貼文+留言******
執行中
spend260.17min
這篇已經沒有更多留言嘍
******已讀取 2666 篇貼文+留言******
執行中
spend260.43min
這篇已經沒有更多留言嘍
******已讀取 2667 篇貼文+留言******
執行中
spend260.7min
這篇已經沒有更多留言嘍
******已讀取 2668 篇貼文+留言******
執行中
spend260.96min
這篇已經沒有更多留言嘍
******已讀取 2669 篇貼文+留言******
執行中
spend261.22min
這篇已經沒有更多留言嘍
******已讀取 2670 篇貼文+留言******
執行中
spend261.48min
這篇已經沒有更多留言嘍
******已讀取 2671 篇貼文+留言******
執行中
spend261.75min
這篇已經沒有更多留言嘍
******已讀取 2672 篇貼文+留言******
執行中
spend262.01min
這篇已經沒有更多留言嘍
******已讀取 2673 篇貼文+留言******
執行中
spend262.27min
這篇已經沒有更多留言嘍
******已讀取 2674 篇貼文+留言******
執行中
spend262.54min
這篇已經沒有更多留言嘍
******已讀取 2675 篇貼文+留言******
執行中
spend262.8min
這篇已經沒有更多留言嘍
******已讀取 2676 篇貼文+留言******
執行中
spend263.06min
這篇已經沒有更多留言嘍
******已讀取 2677 篇貼文+留言******
執行中
spend263.32min
這篇已經沒有更多留言嘍
******已讀取 2678 篇貼文+留言******
執行中
spend263.58min
這篇已經沒有更多留言嘍
******已讀取 2679 篇貼文+留言******
執行中
spend263.85min
這篇已經沒有更多留言嘍
******已讀取 2680 篇貼文+留言******
執行中
spend264.11min

spend296.85min
這篇已經沒有更多留言嘍
******已讀取 2804 篇貼文+留言******
執行中
spend297.11min
這篇已經沒有更多留言嘍
******已讀取 2805 篇貼文+留言******
執行中
spend297.37min
這篇已經沒有更多留言嘍
******已讀取 2806 篇貼文+留言******
執行中
spend297.64min
這篇已經沒有更多留言嘍
******已讀取 2807 篇貼文+留言******
執行中
spend297.9min
這篇已經沒有更多留言嘍
******已讀取 2808 篇貼文+留言******
執行中
spend298.16min
這篇已經沒有更多留言嘍
******已讀取 2809 篇貼文+留言******
執行中
spend298.43min
這篇已經沒有更多留言嘍
******已讀取 2810 篇貼文+留言******
執行中
spend298.69min
這篇已經沒有更多留言嘍
******已讀取 2811 篇貼文+留言******
執行中
spend298.95min
這篇已經沒有更多留言嘍
******已讀取 2812 篇貼文+留言******
執行中
spend299.21min
這篇已經沒有更多留言嘍
******已讀取 2813 篇貼文+留言******
執行中
spend299.48min
這篇已經沒有更多留言嘍
******已讀取 2814 篇貼文+留言******
執行中
spend299.74min
這篇已經沒有更多留言嘍
******已讀取 2815 篇貼文+留言******
執行中
spend300.0min
這篇已經沒有更多留言嘍
******已讀取 2816 篇貼文+留言******
執行中
spend300.26min
這篇已經沒有更多留言嘍
******已讀取 2817 篇貼文+留言******
執行中
spend300.52min
這篇已經沒有更多留言嘍
******已讀取 2818 篇貼文+留言******
執行中
spend300.79min
這篇已經沒有更多留言嘍
******已讀取 2819 篇貼文+留言******
執行中
spend301.05min
這篇已經沒有更多留言嘍
******已讀取 2820 篇貼文+留言******
執行中

spend333.67min
這篇已經沒有更多留言嘍
******已讀取 2944 篇貼文+留言******
執行中
spend333.93min
這篇已經沒有更多留言嘍
******已讀取 2945 篇貼文+留言******
執行中
spend334.19min
這篇已經沒有更多留言嘍
******已讀取 2946 篇貼文+留言******
執行中
spend334.45min
這篇已經沒有更多留言嘍
******已讀取 2947 篇貼文+留言******
執行中
spend334.72min
這篇已經沒有更多留言嘍
******已讀取 2948 篇貼文+留言******
執行中
spend334.98min
這篇已經沒有更多留言嘍
******已讀取 2949 篇貼文+留言******
執行中
spend335.24min
這篇已經沒有更多留言嘍
******已讀取 2950 篇貼文+留言******
執行中
spend335.5min
這篇已經沒有更多留言嘍
******已讀取 2951 篇貼文+留言******
執行中
spend335.77min
這篇已經沒有更多留言嘍
******已讀取 2952 篇貼文+留言******
執行中
spend336.03min
這篇已經沒有更多留言嘍
******已讀取 2953 篇貼文+留言******
執行中
spend336.29min
這篇已經沒有更多留言嘍
******已讀取 2954 篇貼文+留言******
執行中
spend336.55min
這篇已經沒有更多留言嘍
******已讀取 2955 篇貼文+留言******
執行中
spend336.82min
這篇已經沒有更多留言嘍
******已讀取 2956 篇貼文+留言******
執行中
spend337.08min
這篇已經沒有更多留言嘍
******已讀取 2957 篇貼文+留言******
執行中
spend337.34min
這篇已經沒有更多留言嘍
******已讀取 2958 篇貼文+留言******
執行中
spend337.6min
這篇已經沒有更多留言嘍
******已讀取 2959 篇貼文+留言******
執行中
spend337.86min
這篇已經沒有更多留言嘍
******已讀取 2960 篇貼文+留言******
執行中

## 印出貼文+留言

In [21]:
import pandas as pd
import re

pattern= r"[^\\\[\]'\"\^+⋯…「」\［\］«»‹›∠〔｀˙∇(⁎⁍̴̛ᴗ⁍̴̛⁎)‼𝐐△▽○◇□☆▷◁♤♡♢♧♣♦♥°%‰％º²³ⁿ∟∠⊥⊿⊙⊕∧∨∪∩∈∉≌∽∝∞∫∣∆∭≈≥≒≠≡≮≧≦≤≯◀⁽⁾˃▶★◍■⚗︎·̫⚗︎◆✩※░▒▓∀▏▎▍▌▋▊▉㏘㏂☞☛☚☟→←↓↘↖↑↗☜☚✘✔√×☒☑☺☹♂♀☽☾☀☼▼▲☼☀∷〕‿ ᷄˂✧〖〗❛٩(ˊᗜˋ*)و(ง •̀-•́)ง▫️︵˂︶︷︸︺︹︿﹀︾︽﹁﹂﹄,̫,̫,̫ ᷅,️﹢﹣·×±∶÷﹃︻︼︘︗〈〉´༎`🇦🇿🇧🇨🇩🇪🇫🇬🇭🇮🇯🇰🇱🇲🇳🇴🇵🇶🇷🇸🇹🇺🇻🇼🇽🇾－／《％＋·▪‥…▁▂▃▄▅▆▇█∷※•🥲、‘’，【】》˶⁎▫*＊⁍“”〝〞'＂＇＇～~。˵＿_¯￣﹋﹏﹍﹉﹊﹎`ˋ︴¦¡¿ˇ­¨｜|⁻‾.@＠？?『』#＃%&＆$*……￥，\-（）()＜＞﹝﹞<>：；:;/！!,\{\}=＝\w\s]|[\u4e00-\u9fa5]+"
df1 = pd.DataFrame(columns=['siteName', 'postID', 'post'])   

postId= 0
for post in postL:
    time = post.find('time','FH9sR RhOlS').text
    time = re.sub('=', '', time)
    
    # content的網頁碼可能要改
    content =  post.find('span','_7UhW9 xLCgt MMzan KV-D4 se6yk T0kll')
    if content is not None:
        content = ','.join(re.findall(pattern, str(content)))
    
    postId= postId+ 1
    
    df1 = df1.append({
    'siteName': siteName, 
    'postID':postId,
    'time' :time ,
    'post' : content} , ignore_index= True)

In [22]:
df1

,siteName,postID,post,time
0,niusnews,1,"惡搞版習大維尼又來啦,日本人氣插畫家西村雄二先前繪製搞怪又獨特的小熊維尼,被網友拿來做各種迷...",7 分鐘
1,niusnews,2,"精品小包絕對是愛買包的妞妞們最喜愛的包款之一,不僅外型可愛又有經典品牌元素,重點是價格也親民...",22 小時
2,niusnews,3,"那幾天也想要舒適涼爽到底啊,上半身流著汗,💥,下半身陣陣涼意,❄,簡直就是妞編的夏季悶熱救星...",1 天
3,niusnews,4,"很需要涼一下,😂",1 天
4,niusnews,5,"戴口罩用眼神也要狂放電,✨,黛西朵兒,大彩妝單品女孩們必收,新手也可以畫出精緻迷人的溫柔電眼...",1 天
...,...,...,...,...
3052,niusnews,3053,"鍵盤賞楓巡禮,⁣,受到大人小孩喜愛的姆明谷公園,入秋後的楓紅盛況營造出有別於卡通的童稚氛圍,...",72 週
3053,niusnews,3054,"社群丼傑哥,妞編,入門必備設備推薦,⁣,由臉書熱門行銷人社團,社群丼,創辦人,傑哥,與工作團...",72 週
3054,niusnews,3055,"不管啦,有柴有愛心,🐶,🐶,🐶,決定把柴柴們都吃掉,⁣,無論是手工棉花糖厚吐司還是部長鮮奶茶...",72 週
3055,niusnews,3056,"全部都毛茸茸,櫻花妹超大勢,人氣角色毛毛包,大軍來襲,日牌,攜手三麗鷗在,推出超萌手提包,✨...",72 週


In [23]:
path= "AllData/crawler/"
df1.to_excel(path+ f"IG_{siteName}.xlsx")
